## dress will move acording to the head movement

In [12]:
#using as base code
#transparency works
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Load dress images with alpha channel
def load_dress_images(folder_path):
    dress_images = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            dress_images.append(img)
        else:
            print(f'Error loading image: {file_path}')
    return dress_images

# Overlay dress on detected faces
def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)
    dress_x = x + int((w - resized_dress.shape[1]) / 2)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi


# Main function
def main():
    # Folder path containing dress images
    dress_folder_path = r'.\assets'

    # Load dress images
    dress_images = load_dress_images(dress_folder_path)

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


In [3]:
#copy 2 transparency works experi
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
current_dress_index = 0
user_picture_path = r'.\assets'
output_image_paths = []

# Load dress images with alpha channel
def load_dress_images(folder_path):
    dress_images = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            dress_images.append(img)
        else:
            print(f'Error loading image: {file_path}')
    return dress_images


# Overlay dress on detected faces
def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)
    dress_x = x + int((w - resized_dress.shape[1]) / 2)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi


# Main function
def main():
    # Folder path containing dress images
    dress_folder_path = r'.\assets'

    # Load dress images
    dress_images = load_dress_images(dress_folder_path)

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


In [16]:
#transparency works
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'./computer_vision-master/CAFFE_DNN/deploy.prototxt'
caffemodel_path = r'./computer_vision-master/CAFFE_DNN/res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)  # Use 0 for default webcam, or change to the appropriate index

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Load dress images with alpha channel
def load_dress_images(folder_path):
    dress_images = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            dress_images.append(img)
        else:
            print(f'Error loading image: {file_path}')
    return dress_images

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + h  # Set dress y-coordinate below the detected face region
    dress_x = x + int((w - resized_dress.shape[1]) / 5)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress to match the size of the ROI
    resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress[:, :, 3] / 255.0
    inv_dress_mask = 1.0 - dress_mask

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi



# Main function
def main():
    # Folder path containing dress images
    dress_folder_path = './assets'

    # Load dress images
    dress_images = load_dress_images(dress_folder_path)

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()
